# fMRI analysis and visualization - subject averaged data 

#### In this notebook we will do some visualization and analysis of fMRI data which has already been averaged across participants.



In [ ]:
import numpy as np 
from matplotlib import pyplot as plt 
from hdf5storage import loadmat, savemat 
from nilearn import plotting,datasets

In [ ]:
#if you want to keep the code in the github repository
datapath = '/home/ramesh/Teaching/data_archive/hcp_task/'
#use this if you placed this program in the same location as the data. 
#datapath = './'


In [ ]:
regions = np.load(datapath+'regions.npy') # this is the file 
roi_names = regions[:,0] # these are the names of each of 360 roi from the parcellation.
network_names = regions[:,1] # these are the networks each roi "belongs" to
networks = np.unique(regions[:,1]) # these are the unique network names 

#### Load the hcp atlas file.  This file provides a mapping between the 360 roi and the fsaverage (FreeSurfer Average) brain.  fsaverage is the average of 40 brains of healthy individuals.  

In [ ]:
atlas = dict(np.load(datapath+'hcp_atlas.npz'))
fsaverage = datasets.fetch_surf_fsaverage()

In [ ]:
data = loadmat(datapath+'SOCIAL_fmri_subjectaverage.mat')

In [ ]:
data.keys()

In [ ]:
condition_index = data['condition_index']
conditions = data['conditions']
fmri = data['fmri']
nconditions = data['nconditions']
nregions = data['nregions']
nsubjects = data['nsubjects']
subject = data['subject']
task = data['task']

## README 

#### condition_index - index for each data sample, indicating the experimental condition 
#### conditions - conditions in the experiment 
#### fmri - fmri data averaged over participants, nregions x (nsubjects x nconditions)
#### nconditions - number of conditions
#### nregions - number of regions (always 360)
#### nsubjects - number of subjects (always 100)
#### subject - indexes which subject each average comes from. 
#### task - which task the data comes from.  

In [ ]:
print(conditions)

#### For the SOCIAL task 
* #### 'mental' - appearance of social interaction 
* #### 'rnd' - appears to be random 



#### 1. Compute the average of the data in each condition.  Make a plot on the brain of the average data in each condition.   

In [ ]:
avg_fmri = dict()
avg_fmri['mental'] = np.mean(fmri[:,condition_index == 0],axis = 1)
avg_fmri['rnd'] = np.mean(fmri[:,condition_index == 1],axis = 1)


### 

In [ ]:
plot_condition = 'rnd'
plot_data = avg_fmri[plot_condition]
surf_label = plot_data[atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

In [ ]:
surf_label = plot_data[atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

In [ ]:
plot_condition = 'mental'
plot_data = avg_fmri[plot_condition]
surf_label = plot_data[atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

In [ ]:
surf_label = plot_data[atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

#### 2.  Standardize the data using z-score, and compute the different between conditions. Make a bar graph of the difference versus roi. Print out the roi_names and network_names of the 5 regions where activity in `mental` is most higher than `rnd`.  Print out the roi_names and network_names of the 5 regions where `rnd` is most higher than `mental`

In [ ]:
from scipy.stats import zscore
z = zscore(fmri)
#z = zscore(fmri,axis = 0)
diff = np.mean((z[:,condition_index == 0]-z[:,condition_index == 1]),axis = 1)

In [ ]:
plt.figure(figsize = (8,8))
plt.bar(np.arange(nregions),diff)
plt.xlabel('ROI')
plt.ylabel('MENTAL-RANDOM (standard deviation units)')
plt.show()

In [ ]:
difforder = np.argsort(diff)  # python sorting is always ascending.   
rndlarge = difforder[0:5] #so the first 5 here are big negative numbers where rnd is bigger than mental
mentallarge = difforder[-5:] # the last 5 numbers are big positive numbers where mental is bigger than rnd

In [ ]:
print('5 regions with mental larger')
for j in range(5):
    print('ROI: ', roi_names[mentallarge[j]], 'Network: ', network_names[mentallarge[j]])
print('5 regions with rnd larger')
for j in range(5):
    print('ROI: ', roi_names[rndlarge[j]], 'Network: ', network_names[rndlarge[j]])


#### 3. Make a brain map of the difference between `mental` and `rnd` 

In [ ]:
plot_condition = 'mental - rnd'
plot_data = diff
surf_label = plot_data[atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

In [ ]:
surf_label = plot_data[atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

#### 4.  Make a PCA model of the data, and make a bar graph of the difference in loadings between `mental` and `rnd` versus component. Identify and make a brain map of the components which has the highest loadings for `mental` > `'rnd` and the component the highest loadings for `rnd` > `mental` 

In [ ]:
from sklearn.decomposition import PCA
fmriPCA = PCA()
fmriPCA.fit(np.transpose(z))
loadings = fmriPCA.transform(np.transpose(z))
components = fmriPCA.components_
ncomponents = fmriPCA.n_components_
percentvariance = 100*fmriPCA.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(np.arange(ncomponents),percentvariance)
plt.xlabel('Component')
plt.ylabel('Percent Variance')
plt.show()

In [ ]:
c_mental = np.mean(loadings[condition_index == 0,:],axis = 0)
c_rnd = np.mean(loadings[condition_index == 1,:], axis = 0)
c_diff = c_mental-c_rnd
plt.bar(np.arange(ncomponents),c_diff)
plt.xlabel('Component')
plt.ylabel('Mental-Random (sd units)')
cmax_mental = np.argmax(c_diff)
cmax_rnd = np.argmin(c_diff)

In [ ]:
surf_label = components[cmax_rnd,:][atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = 'cmax_rnd',black_bg = True, vmax = 0.2)

In [ ]:
surf_label = components[cmax_rnd,:][atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = 'cmax_rnd',black_bg = True, vmax = 0.2)

In [ ]:
surf_label = components[cmax_mental,:][atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = 'cmax_mental',black_bg = True, vmax = 0.2)

In [ ]:

surf_label = components[cmax_mental,:][atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = 'cmax_mental',black_bg = True, vmax = 0.2)